In [ ]:
# Importing Libraries
import tables
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('/Users/eframe/dmi/src')
import compton 
import codedAperture as ca
import eventAnalysis as ea
import calibrate as calib
import matplotlib.ticker as ticker
import pandas as pd
from scipy.optimize import curve_fit
from scipy.special import xlogy, erf
import math as m

In [ ]:
# Inputs for Compton image reconstruction
E, sigma = 440, 2
mu_low, mu_high = -0.4, 1
lever_thres = 14
poses = np.array( [ 0, 315, 270, 225, 180, 135, 90, 45 ] ) * np.pi / 180
inputs = np.array( [ '/Users/eframe/Downloads/ac2pt_top_COMP.h5'] )
binSize = 2
sourceX, sourceY, sourceZ = np.mgrid[ -29:31:binSize, -29:31:binSize, -29:31:binSize ]
sourcePixels = np.array( [ sourceX.flatten(), sourceY.flatten(), sourceZ.flatten() ] ).T

In [ ]:
# Getting Compton cones from detector data for each pose
interPos, coneDir, coneMu = [], [], []
for i in np.arange( len( inputs ) ):
    f = tables.open_file( inputs[ i ], 'r' )
    time = f.root.EventData.read()['timestamp']
    duration = ( time[-1] * 10 ** -8 - time[0] * 10 ** -8 ) / 60
    print('minutes:', duration)
    d = f.root.Interactions.Double.read()
    d['x'] = d['x'] - 38 
    d['y'] = d['y'] - 38  
    l22 = d.reshape( int ( len ( d ) / 2 ), 2 ) 

    # Getting Cone Data
    emask = ( l22['energy'].sum( axis = 1 ) >= E - sigma ) & ( l22['energy'].sum( axis = 1 ) <= E + sigma )   
    events_old = l22[emask]
    events = compton.correct_depth( events_old )
    events['z'] = -events['z']
    
    iP, iP2, cD, cM = compton.convertToConeData( events, E, lever_thres )
    
    # Sequencing Interactions
    energy1 = np.array( [ events['energy' ][ :, 0 ], events['energy'][ :, 1 ] ] ).T
    energy2 = np.array( [ events['energy' ][ :, 1 ], events['energy'][ :, 0 ] ] ).T
    p1 = np.squeeze( np.array( [ events['x'][ :, 0 ], events['y'][ :, 0 ], events['z'][ :, 0 ] ] ) ).T
    p2 = np.squeeze( np.array( [ events['x'][ :, 1 ], events['y'][ :, 1 ], events['z'][ :, 1 ] ] ) ).T
    pos1 = np.concatenate( ( [ p1, p2 ] ) , axis = 1 ).reshape( len( p1 ), 2, 3 )
    pos2 = np.concatenate( ( [ p2, p1 ] ) , axis = 1 ).reshape( len( p1 ), 2, 3 )

    P12, P21 = compton.sequence_probability( energy1, pos1, E )
    mask = ( P12 < P21 )
    energy = energy1.copy()
    energy[mask] = energy2[mask]
    pos = pos1.copy()
    pos[mask] = pos2[mask]
    iP, iP2, cD, cM = compton.convertToConeData_SEQ( energy, pos, E, lever_thres )
    
    mask = ( cM > mu_low ) & ( cM < mu_high )
    interPos.append( iP[mask] )
    coneMu.append( cM[mask] )
    coneDir.append( cD[mask] )

In [ ]:
# Heatmap of interaction locations
%matplotlib inline
for i in np.arange( len( interPos ) ):
    print( poses[i] )
    pos = interPos[i]
    plt.figure( figsize=( 6,12 ) )
    # XY
    plt.subplot( 311 )
    plt.hist2d( pos[ :, 0 ], pos[ :, 1 ], bins = ( 38, 38 ) )
    plt.colorbar()
    plt.xlabel( "X (mm)" )
    plt.ylabel( "Y (mm)" )
    # XZ
    plt.subplot( 312 )
    plt.hist2d(pos[ :, 0 ], pos[ :, 2 ], bins = ( 38, 38 ) )
    plt.colorbar()
    plt.xlabel( "X (mm)" )
    plt.ylabel( "Z (mm)" )
    # YZ
    plt.subplot( 313 )
    plt.hist2d(pos[ :, 1 ], pos[ :, 2 ], bins = ( 38, 38 ) )
    plt.colorbar()
    plt.xlabel( "Y (mm)" )
    plt.ylabel( "Z (mm)" )
    # Render
    plt.tight_layout()
    plt.show()

In [ ]:
# Defining new source image space for each pose
center = np.array( [ 0, 0, 30] )
sourcePixelsNew = []
for i in np.arange( len( poses ) ):
    ang = poses[ i ]
    R = np.array( [  [ np.cos( ang ), 0, np.sin( ang ) ], 
                   [ 0, 1, 0 ], 
                   [ -np.sin( ang ), 0, np.cos( ang ) ] ] ).T
    B = np.array( [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ] )
    K = np.array( np.dot( R, B ) )
    sourcePixelsNew.append(  np.dot( sourcePixels, R ) + center ) 

In [ ]:
# Getting Backprojected Data
%matplotlib inline
backproj = []
position1 = []
mu1 = []
for i in np.arange( len( interPos ) ):
    print( poses[i] * 180 / np.pi )
    bp = compton.coneVoxel2( sourcePixelsNew[i], interPos[i], \
                            coneDir[i], coneMu[i], 0.1, binSize  ) 
    v = ( bp ).sum( 1 )
    mask2 = np.isnan( v ) | ( v < 1e-20 )
    backproj.append( bp[~mask2] )
    position1.append( interPos[i][~mask2] )
    mu1.append( coneMu[i][~mask2])
    hist, bins = np.histogram( v[~mask2], bins = 100 )
    plt.plot( bins[ :-1 ], hist )
    plt.show()

In [ ]:
hist, bins = np.histogram( mu, bins = 100 )
plt.plot(bins[:-1], hist)
plt.show()

In [ ]:
R2[j].shape

In [ ]:
j = 201
data = sysresponse[j] * R2[j]
print(position[j], mu[j])
outfile = '/Users/eframe/Downloads/bp2.h5'
f = tables.open_file( outfile, 'w' )
f.create_array( '/', 'image', data.flatten() )
f.close()

In [ ]:
# Fraction of Back-projected Cone in Edge Region
det, thick = 2900, 2.5
backproj_new = backproj.copy()
fractions = []
for p in np.arange( len(interPos ) ):
    edge_region = ( sourcePixelsNew[p][:, 0] < -( det - thick ) ) |\
                  ( sourcePixelsNew[p][:, 0] > ( det - thick ) ) |\
                  ( sourcePixelsNew[p][:, 1] < -( det - thick ) ) |\
                  ( sourcePixelsNew[p][:, 1] > ( det - thick ) ) |\
                  ( sourcePixelsNew[p][:, 2] - center[2] < -( det - thick ) ) |\
                  ( sourcePixelsNew[p][:, 2] - center[2] > ( det - thick ) ) 
    for i in np.arange( len( backproj_new[p] ) ):
        fraction = backproj_new[p][i][edge_region].sum() / ( backproj_new[p][i].sum() + 1e-10 )
        fractions.append( fraction )
        backproj_new[p][i][edge_region] = 0

In [ ]:
# Histogram of Back-projected Cone Fractions in Edge Region
hist, bins = np.histogram( fractions, bins = 10 )
plt.plot( bins[:-1], hist )
plt.xlabel( 'Fraction of Cone in Edge Region' )
plt.ylabel( '# of Compton Cones' )
plt.show()

In [ ]:
# Getting Rid of Zero Values Events for Backprojected Data
backproj_New = []
for i in np.arange( len( interPos ) ):
    vals = ( backproj_new[i] ).sum( 1 )
    mask = ( vals > 0 ) 
    backproj_New.append( backproj_new[i][mask] )
    hist, bins = np.histogram( vals[mask], bins = 100 )
    plt.plot( bins[ :-1 ], hist )
    plt.show()

In [ ]:
# Calculating Sensitivity Map 1
sensMap1 = []
det = 35
for i in np.arange( len( interPos ) ):
    x = np.round( sourcePixelsNew[i][:,0] ).astype(int)
    y = np.round( sourcePixelsNew[i][:,1] ).astype(int)
    z = np.round( sourcePixelsNew[i][:,2] ).astype(int)
    c1 = x + det
    c2 = x - det
    d1 = y + det
    d2 = y - det
    a1 = c1 / np.sqrt( c1 ** 2 + z ** 2 )
    b1 = d1 / np.sqrt( d1 ** 2 + z ** 2 )
    a2 = c2 / np.sqrt( c2 ** 2 + z ** 2 )
    b2 = d2 / np.sqrt( d2 ** 2 + z ** 2 )
    firstsum = np.arcsin( a1 * b1 ) * ( ( -1 ) ** 2 ) + np.arcsin( a1 * b2 ) * ( ( -1 ) ** 3 )
    secondsum = np.arcsin( a2 * b1 ) * ( ( -1 ) ** 3 ) + np.arcsin( a2 * b2 ) * ( ( -1 ) ** 4 )
    sens1 =  ( firstsum + secondsum ) / ( 4 * np.pi ) * 0.012 
    sensMap1.append( sens1 )

In [ ]:
# Calculating Sensitivity Map 2
sensMap2 = []
for i in np.arange( len( interPos ) ):
    sens2 = backproj_New[i].sum(0)
    sensMap2.append( sens2 )

In [ ]:
# Sensitivity 1 Map
%matplotlib inline
S1 = np.array(sensMap1).sum(0)

# sens_mask = ( sourcePixelsNew[0][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[1][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[2][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[3][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[4][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[5][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[6][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[7][:,2] < 0.5 )
# S1[sens_mask] = 0

sens_grid = S1.reshape(sourceX.shape)
a, b = min(S1.flatten()), max(S1.flatten())
for i in np.arange( sourceX.shape[1] ):
    fig, ax = plt.subplots()
    grid = sens_grid[ :, :, i] 
    im = ax.pcolormesh( grid, cmap='jet', vmin=a, vmax=b, shading = 'gouraud' ) 
    ax.set_xlabel( 'x (mm)', fontsize = 20 )
    ax.set_ylabel( 'z (mm)', fontsize = 20 )
    ax.tick_params( labelsize = 20 )
    cbar = plt.colorbar( im )
    cbar.set_label( label = 'intensity', rotation = 270, fontsize = 15, labelpad = 20 )
    cbar.ax.tick_params( labelsize = 20 )
    plt.show()

In [ ]:
# Sensitivity 2 Map
%matplotlib inline
S2 = np.array(sensMap2).sum(0)

# sens_mask = ( sourcePixelsNew[0][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[1][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[2][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[3][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[4][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[5][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[6][:,2] < 0.5 ) |\
#             ( sourcePixelsNew[7][:,2] < 0.5 )
# S1[sens_mask] = 0

sens_grid = S2.reshape(sourceX.shape)
a, b = min(S2.flatten()), max(S2.flatten())
for i in np.arange( sourceX.shape[1] ):
    fig, ax = plt.subplots()
    grid = sens_grid[ :, :, i] 
    im = ax.pcolormesh( grid, cmap='jet', vmin=a, vmax=b, shading = 'gouraud' ) 
    ax.set_xlabel( 'x (mm)', fontsize = 20 )
    ax.set_ylabel( 'z (mm)', fontsize = 20 )
    ax.tick_params( labelsize = 20 )
    cbar = plt.colorbar( im )
    cbar.set_label( label = 'intensity', rotation = 270, fontsize = 15, labelpad = 20 )
    cbar.ax.tick_params( labelsize = 20 )
    plt.show()

In [ ]:
# Histogram of Sensitivities
hist1, bins1 = np.histogram( S1, bins = 100 )
hist2, bins2 = np.histogram( S2, bins = 100 )
plt.plot( bins1[:-1], hist1, 'k', label='S1')
plt.legend()
plt.show()
plt.plot( bins2[:-1], hist2, 'r', label='S2')
plt.legend()
plt.show()

In [ ]:
nIter = 5
lamb = np.ones( len( sourcePixels ) )
eps = 0.05
sens = S1.copy()
for i in np.arange( nIter ):
    ratio = np.zeros( len( sourcePixels ) ) 
    for p in np.arange( 0, len( interPos ), 1 ):
        print( 'iteration: %i' %i, 'pose: %i' %p )
        sysMat = backproj_New[p]
        projExpected = np.dot( sysMat, lamb ) 
        frac = np.divide( sysMat.T, projExpected, out = np.zeros_like( sysMat.T ), where = projExpected != 0 )
        ratio = ratio + frac.sum(1) 
    lamb = lamb * ratio * sens / ( sens ** 2 + max( sens ) ** 2 * eps ** 2 )

In [ ]:
# Compton Image Reconstruction
%matplotlib inline
depth = np.array( sourceZ[:,0][0] )[:] + center[2]
xmin, xmax = sourceX[:,0][:,0][0], sourceX[:,0][:,0][-1] + 1
ymin, ymax = sourceY[0,:][:,0][0], sourceY[0,:][:,0][-1] + 1
X, Y = np.mgrid[  xmin:xmax:binSize, ymin:ymax:binSize ]

lamb2 = lamb.copy()

lamb2 = np.fliplr( lamb2.reshape(sourceX.shape)[:,:,:] )
a, b = min(lamb2.flatten()), max(lamb2.flatten())
for i in np.arange( len(depth) ):
    fig, ax = plt.subplots()
    grid = lamb2[ :, :, i] 
    im = ax.pcolormesh( X, Y, grid, cmap='jet', vmin=a, vmax=b, shading = 'gouraud' ) 
    ax.set_xlabel( 'x (mm)', fontsize = 20 )
    ax.set_ylabel( 'z (mm)', fontsize = 20 )
    ax.tick_params( labelsize = 20 )
    cbar = plt.colorbar( im )
    cbar.set_label( label = 'intensity', rotation = 270, fontsize = 15, labelpad = 20 )
    cbar.ax.tick_params( labelsize = 20 )
    plt.title( depth[i] )
    plt.show()

In [ ]:
num=0
for i in np.arange( len( interPos ) ):
    num = num + backproj_New[i].shape[0]
num

In [ ]:
lamb3 = lamb.copy() * sens * num  / np.sum( lamb.copy() * sens )
hist, bins = np.histogram(lamb3.flatten(), bins=100, range=(0,2))
plt.plot(bins[:-1], hist)
plt.ylim(0, 100)
plt.show()

In [ ]:
lamb3.sum()

In [ ]:
outfile = '/Users/eframe/Downloads/mouse11.3.h5'
f = tables.open_file( outfile, 'w' )
f.create_array( '/', 'image', lamb2.flatten() )
f.close()

In [ ]:
# Rotation axis for mouse measurement on 10.6.21: 0, 0, 38

In [ ]:
# tot = 0
# for i in np.arange( len( backproj_new ) ):
#     num = backproj_new[i].shape[0]
#     print(num)
#     tot = num + tot
# print(tot)

In [ ]:
# # Loading Sensitivity Map
# senstable = pd.read_hdf( '/Users/eframe/dmi/sensMap662keV.h5', 'vals' )

# # Getting new sensitivity values for each pose
# sensMap = np.zeros( ( len( poses ), len(sourcePixels) ) )
# for i in np.arange( len( poses ) ):
#     print( poses[i] * 180 / np.pi )
#     for j in np.arange( sourcePixels.shape[0] ):
#         x = np.round( sourcePixelsNew[i][ j, 0 ] ).astype(int)
#         y = np.round( sourcePixelsNew[i][ j, 1 ] ).astype(int)
#         z = np.round( sourcePixelsNew[i][ j, 2 ] ).astype(int)
#         ix = 100 * 201 * ( 101 + x ) 
#         iy = 100 * ( 101 + y )
#         iz = z 
        
#         if ( iz > 0 ) & ( iz >= ( sourcePixels[:,2] + center[2] ).min() ):
#             sensMap[i][j] = senstable[ix - ( 100 * 201 ):ix][iy - 101:iy][iz:iz+1]['sens'].values 
#         else:
#             sensMap[i][j] = 0 

In [ ]:
# # Loading Sensitivity Map
# senstable = pd.read_hdf( '/Users/eframe/dmi/sensMap440keV.h5', 'vals' )

# # Getting new sensitivity values for each pose
# sensMap = np.zeros( ( len( poses ), len(sourcePixels) ) )
# for i in np.arange( len( poses ) ):
#     print( poses[i] * 180 / np.pi )
#     for j in np.arange( sourcePixels.shape[0] ):
#         x = np.round( sourcePixelsNew[i][ j, 0 ] ).astype(int)
#         y = np.round( sourcePixelsNew[i][ j, 1 ] ).astype(int)
#         z = np.round( sourcePixelsNew[i][ j, 2 ] ).astype(int)
#         ix = 100 * 45 * ( 23 + x ) 
#         iy = 100 * ( 23 + y )
#         iz = z 
        
#         if ( iz > 0 ) & \
#         ( z >= ( sourcePixels[:,2] + center[2] ).min() ) & \
#         ( z <= ( sourcePixels[:,2] + center[2] ).max() ) & \
#         ( x >= ( sourcePixels[:,0] ).min() ) & \
#         ( x <= ( sourcePixels[:,0] ).max() ):
#             sensMap[i][j] = senstable[ix - ( 100 * 45 ):ix][iy - 100:iy][iz-1:iz]['sens'].values
#         else:
#             sensMap[i][j] = 0 

In [ ]:
# # Calculating Sensitivity Map 1
# sensMap = []
# det = 35
# for i in np.arange( len( poses ) ):
#     x = np.round( sourcePixelsNew[i][:,0] ).astype(int)
#     y = np.round( sourcePixelsNew[i][:,1] ).astype(int)
#     z = np.round( sourcePixelsNew[i][:,2] ).astype(int)
#     R = np.sqrt(x**2+y**2+z**2)
#     c1 = x + det
#     c2 = x - det
#     d1 = y + det
#     d2 = y - det
#     a1 = c1 / np.sqrt( c1 ** 2 + z ** 2 )
#     b1 = d1 / np.sqrt( d1 ** 2 + z ** 2 )
#     a2 = c2 / np.sqrt( c2 ** 2 + z ** 2 )
#     b2 = d2 / np.sqrt( d2 ** 2 + z ** 2 )
#     firstsum = np.arcsin( a1 * b1 ) * ( ( -1 ) ** 2 ) + np.arcsin( a1 * b2 ) * ( ( -1 ) ** 3 )
#     secondsum = np.arcsin( a2 * b1 ) * ( ( -1 ) ** 3 ) + np.arcsin( a2 * b2 ) * ( ( -1 ) ** 4 )
#     sens =  ( firstsum + secondsum ) / ( 4 * np.pi ) * 0.012 
#     radius = np.sqrt( x ** 2 + ( z + center[2] ) ** 2 )
#     mask = ( x < sourcePixels[:,0].min() ) | ( z < sourcePixels[:,2].min() + center[2]  ) 
#     sens[mask] = 0
#     mask = ( x > sourcePixels[:,0].max() ) | ( z > sourcePixels[:,2].max() + center[2]  )
#     sens[mask] = 0
#     mask = ( z < 0 )
#     sens[mask] = 0
#     sensMap.append( sens )

In [ ]:
#     while sumtot < int( len(backproj[i]) /  ):
#         sumtot = np.sum( hi[i][-j:] )
#         j = j + 1
#     mask = ( vals[i] > bi[i][-j] ) 

In [ ]:
# interPos = np.concatenate((interPos[0], interPos[1]))
# coneMu = np.concatenate((coneMu[0], coneMu[1]))
# coneDir = np.concatenate((coneDir[0], coneDir[1]))
# interPos = np.array([interPos])
# coneMu = np.array([coneMu])
# coneDir = np.array([coneDir])